<a href="https://colab.research.google.com/github/aldojda/Processing-DataClassification/blob/main/00_WebScrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests beautifulsoup4

import re

In [ ]:
def calification_to_row(soup, id):
    # Extract the relevant data
    date = soup.find('div', class_='date').get_text(strip=True)
    try:
        try:
            general_quality = soup.find('span', class_='score bueno').get_text(strip=True)
            ease = soup.find_all('span', class_='score bueno')[1].get_text(strip=True)

        except:
            general_quality = soup.find('span', class_='score malo').get_text(strip=True)
            ease = soup.find_all('span', class_='score malo')[1].get_text(strip=True)
    except:
        general_quality = None
        ease = None
    try:
        class_name = soup.find('span', class_='name').find('span', class_='response').get_text(strip=True)
    except: class_name = None
    try:
        attendance = soup.find('span', class_='attendance').find('span', class_='response').get_text(strip=True)
    except:  attendance = None
    try:
        received_grade = soup.find_all('span', class_='grade')[0].find('span', class_='response').get_text(strip=True)
    except:  received_grade = None
    try:
        interest = soup.find_all('span', class_='grade')[1].find('span', class_='response').get_text(strip=True)
    except: interest = None
    try:
        comments = soup.find('p', class_='commentsParagraph').get_text(strip=True)
    except: comments = None
    try:
        helpful_count = soup.find('a', class_='votar_icon helpful').find('span', class_='count').get_text(strip=True)
    except:  helpful_count =  None
    try:
        not_helpful_count = soup.find('a', class_='votar_icon nothelpful').find('span', class_='count').get_text(strip=True)
    except: not_helpful_count = None

    # Create a dictionary to store the data
    data = {
        'Date': date,
        'General Quality': general_quality,
        'Ease': ease,
        'Class Name': class_name,
        'Attendance': attendance,
        'Received Grade': received_grade,
        'Interest': interest,
        'Comments': comments,
        'Helpful Count': helpful_count,
        'Not Helpful Count': not_helpful_count,
        'id': id
    }
    return data

In [ ]:
def get_dataframe_profesoresURL(url, escuela):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find_all('tbody', class_='odd')
    # Find the script tag containing the variable
    script_tag = soup.find('script', string=re.compile(r'var dataSet = (.*?);'))

    if script_tag:
        # Extract the content between "var dataSet =" and "$"
        match = re.search(r'var dataSet = (.*?)\$',
                        script_tag.text, re.DOTALL)

        data = match.group(1)
    # evaluamos el string eliminando los ultimos 2 elementos [lista de diccionarios de datos]
    list_dicts = eval(data[:-2].replace("null", "None"))
    print("Cantidad de profesores :",  len(list_dicts))
    # generamos lista de urls
    urls = [profesor_url(i) for i in  list_dicts]
    # almacenamos como dataframe
    df_profesores = pd.DataFrame(list_dicts)
    # guardamos url
    df_profesores["url"] = urls
    df_profesores["escuela"] = escuela

    return df_profesores

In [ ]:
#funciones para reemplazar carácteres para poder realizar la consulta
def replace_latin_chars(s):
    translate_re = re.compile(r'[ÁÉÍÓÚÑÜáéíóúñü]')
    translate = {
        "Á": "A", "É": "E", "Í": "I", "Ó": "O", "Ú": "U", "Ñ": "N", "Ü": "U",
        "á": "a", "é": "e", "í": "i", "ó": "o", "ú": "u", "ñ": "n", "ü": "u"
    }
    return re.sub(translate_re, lambda match: translate[match.group(0)], s)

def fix_url(string):
    nombre = string.strip()
    nombre = replace_latin_chars(nombre)
    nombre = re.sub(r'[^a-zA-Z0-9\s]', '', nombre)
    nombre = nombre.replace("(", "")
    nombre = nombre.replace(")", "")
    nombre = nombre.replace(" -", "-")
    nombre = nombre.replace("- ", "-")
    nombre = nombre.replace("\n", "")
    nombre = nombre.replace("\r", "")
    nombre = nombre.replace("  ", " ")
    nombre = nombre.replace("\\", "")
    nombre = nombre.replace(" ", "-")
    return nombre

def escuela_url(nombre, id):
    return "/escuelas/" + fix_url(nombre) + "_" + id

#función que permite encontrar el url asociado
def profesor_url(dict_):
    nombre = fix_url(dict_["n"])
    apellido = fix_url(dict_["a"])
    return  "https://www.misprofesores.com/profesores/" + nombre + "-" + apellido + "_" + dict_["i"]


#funicones para la consulta y proceso de generación de los datos
def materia(mat):
    datos = [i.text for i in mat]
    mat = datos[::4]
    obl = datos[1::4]
    calif_al = datos[2::4]
    interes = datos[3::4]
    datos = {"materia": mat,"obligatoria": obl, "calificacion_alumno": calif_al, "interes": interes }
    return datos

def score_tab():
    dates = []
    score1 = []
    score2 = []
    for i in soup.find_all('td', class_ = "rating"):
        date = [j.text  for j in   i.find_all("div", class_ = "date")]
        score = [j.text  for j in   i.find_all("span", class_ = ["score bueno",  "score regular", "score malo"])]
        dates.append(date[0])
        score1.append(score[0])
        score2.append(score[1])
    return dates, score1, score2
#funcion que permite extraer los datos de la url seleccionada
def extraerDatos(url):
    global soup
    df = pd.DataFrame()
    comments = [1]
    #si hay mas paginas iteramos en aquellas que tengan más páginas
    for i in range(1,20):
        if i >1:
            r = requests.get(url + "?pag=" + str(i), verify=False)
            if len(comments) == 0 :
                #print("saliendo del ciclo", df.shape)
                return df
        else:
            r = requests.get(url, verify=False)
        #consultamos información de la página web
        soup = BeautifulSoup(r.content,  "lxml")
        datos = materia(soup.find_all('span', class_='response'))
        dates, score1, score2 = score_tab()
        comments = [i.text.replace('\n', '') for i in soup.find_all('p', class_='commentsParagraph')]
        rating = [i.text for i in soup.find_all('span', class_='rating-type')]
        datos["commentario"] = comments
        datos["calidad_general"] = score1
        datos["facilidad"] = score2
        datos["date"] = dates
        try:
            datos = pd.DataFrame(datos)
            df = pd.concat([df, datos])
        except:
            #print(datos)
            print("error")

In [ ]:
## Ciclo para iterar sobre urls DE DISTINTAS DEPENDENCIAL DE LA UANL
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.misprofesores.com/Buscar?buscar=Escuelas&buscar_texto=UANL"

# Send a GET request to the page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', class_='table table-striped buscar_table')
    rows = table.find_all('tr')
    data = []
    # iteramos por cada row para extraer los datos y urls
    for n, row in enumerate(rows[1:]):
        # generamos la información de los rows a exepción del url
        data_row = [value.get_text(strip=True) for value in row]
        # extraemos el url
        url = row.find_all('td')[0].find_all('a', href = True)[0]['href']
        # agregamos el url a row
        data_row.append(url)
        # con el row listo agregamos a la tabla data
        data.append(data_row)
    # guardamos como dataframe
    df_escuelas_uanl = pd.DataFrame(data, columns = ["Escuela", "Municipio", "Estado","num_prof", "url"])
    display(df_escuelas_uanl.head(1))

,Escuela,Municipio,Estado,num_prof,url
0,UANL (Facultad de Medicina),Monterrey,Nuevo Leon,186,https://www.misprofesores.com/escuelas/UANL-Fa...


In [ ]:
# generamos todos los URLs para obtener la información de todos los maestros
df_escuelas_uanl = df_escuelas_uanl[df_escuelas_uanl.num_prof.astype(int) > 5]
list_df_profesores = []
# iteramos sobre las urls de las escuelas
for url_escuela, escuela  in zip(df_escuelas_uanl.url, df_escuelas_uanl.Escuela):
    print(url_escuela)
    # aplicamos funcion para obtener dataframe con información de maestros
    df_profes = get_dataframe_profesoresURL(url_escuela, escuela)
    # guardamos informacion en una lista
    list_df_profesores.append(df_profes)
# convertimos la lista a dataframe
base_maestrosURL = pd.concat(list_df_profesores)

https://www.misprofesores.com/escuelas/UANL-Facultad-de-Medicina_1448
Cantidad de profesores : 186
https://www.misprofesores.com/escuelas/Preparatoria-N-22-UANL_4099
Cantidad de profesores : 93
https://www.misprofesores.com/escuelas/UANL-Preparatoria-2_1362
Cantidad de profesores : 18
https://www.misprofesores.com/escuelas/UANL-Facultad-de-Odontologia_1555
Cantidad de profesores : 91
https://www.misprofesores.com/escuelas/UANL-PREPARATORIA-22_3913
Cantidad de profesores : 26
https://www.misprofesores.com/escuelas/FACULTAD-DE-MUSICA-UANL_3769
Cantidad de profesores : 67
https://www.misprofesores.com/escuelas/Preparatoria-7-Oriente-UANL_6601
Cantidad de profesores : 23
https://www.misprofesores.com/escuelas/UANLFACDYC_3602
Cantidad de profesores : 13
https://www.misprofesores.com/escuelas/Facultad-de-Ciencias-de-la-Comunicacion-UANL_3312
Cantidad de profesores : 110
https://www.misprofesores.com/escuelas/Facultad-de-Agronomia-UANL_3299
Cantidad de profesores : 14
https://www.misprofesore

In [ ]:
base_maestrosURL.head(1)

,i,n,a,d,m,c,url,escuela
0,38428,malena,---,fisiologia,4,6.2500,https://www.misprofesores.com/profesores/malen...,UANL (Facultad de Medicina)


In [ ]:
#ciclo para iterar por url de maestros:
list_rows = []
for id, url in zip(base_maestrosURL.i,  base_maestrosURL.url) :
    # se esta accediendo a la información del maestro url
    print(url)
    # solicitud de datos de la url
    response = requests.get(url)
    # calcular la cantidad de ciclos por maestro:
    pattern = r"https://[^\s]*\?pag=\d+"
    # Find all matches in the HTML content
    urls = re.findall(pattern, response.text)
    # se almacena la información de los comentarios en rows
    rows = BeautifulSoup(response.text, 'html.parser').findAll('tr')[1:]
    # se itera comentario a comentario para dar formato de DataFrame
    for n, row in enumerate(rows):
        # se crea row con la información
        data = calification_to_row(row, id)
        # se agreega la información a lista de calificaciones
        list_rows.append(data)
    if len(urls) > 0:
        for url_n in urls[:-1]:
            # realizamos la consulta de los demás urls
            print("\t", url_n)
            # realizamos la consulta de la ifnormación
            response = requests.get(url_n)
            # encontramos los objetos tr donde se encuentran las calificaciones
            rows = BeautifulSoup(response.text, 'html.parser').findAll('tr')[1:]
            # iteramos por cada uno de los objetos
            for n, row in enumerate(rows):
                # convertimos a diccionario
                data = calification_to_row(row, id)
                # almacenamos
                list_rows.append(data)

Se han truncado las últimas 5000 líneas del flujo de salida.
	 https://www.misprofesores.com/profesores/Ing-Carlos-Rodriguez_81097?pag=4
	 https://www.misprofesores.com/profesores/Ing-Carlos-Rodriguez_81097?pag=5
	 https://www.misprofesores.com/profesores/Ing-Carlos-Rodriguez_81097?pag=6
https://www.misprofesores.com/profesores/Antonio-Rodriguez_103619
https://www.misprofesores.com/profesores/Ing-Julia-Isela-Rodriguez-Carrillo_98280
	 https://www.misprofesores.com/profesores/Ing-Julia-Isela-Rodriguez-Carrillo_98280?pag=2
https://www.misprofesores.com/profesores/LUCIO-ESAU-RODRIGUEZ-DAVALOS_87483
	 https://www.misprofesores.com/profesores/LUCIO-ESAU-RODRIGUEZ-DAVALOS_87483?pag=2
	 https://www.misprofesores.com/profesores/LUCIO-ESAU-RODRIGUEZ-DAVALOS_87483?pag=3
https://www.misprofesores.com/profesores/Miguel-Angel-COMINO-Rodriguez-Enriquez_79828
	 https://www.misprofesores.com/profesores/Miguel-Angel-COMINO-Rodriguez-Enriquez_79828?pag=2
https://www.misprofesores.com/profesores/Jesus-Ro

In [ ]:
comments = pd.DataFrame(list_rows).to_csv("calificaciones_maestros.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
comments = pd.DataFrame(list_rows)

In [ ]:
comments.to_csv("drive/MyDrive/01 - Maestría Ciencia de Datos/00_comentarios_misprofesores.csv")

In [ ]:
base_maestrosURL.to_csv("drive/MyDrive/01 - Maestría Ciencia de Datos/01_maestrosURL.csv")

In [ ]:
df_escuelas_uanl.to_csv("drive/MyDrive/01 - Maestría Ciencia de Datos/02_escuelasURL.csv")